# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [25]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('assets/spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [26]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [27]:
def answer_one():
    total = 0
    for a in spam_data['target']:
        if a == 1:
            total += 1
    
    return total / len(spam_data['target']) * 100

answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [74]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
def answer_two():
    x = CountVectorizer().fit(X_train)
    features = np.array(x.get_feature_names())
    aux = list(map(len, features))
    return features[np.argmax(aux)]
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [75]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    X = CountVectorizer().fit(X_train)
    X_aux = X.transform(X_train)
    model = MultinomialNB(alpha = 0.1)
    model.fit(X_aux, y_train)
    predict = model.predict(X.transform(X_test))
    return roc_auc_score(y_test, predict)

answer_three()

0.9720812182741116

vector### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters. The transformed data will be a compressed sparse row matrix where the number of rows is the number of documents in `X_train`, the number of columns is the number of features found by the vectorizer in each document, and each value in the sparse matrix is the tf-idf value. First find the **max** tf-idf value for every feature.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf among the **max** tf-idf values?

Put these features in two series where each series is sorted by tf-idf value. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. Any entries with identical tf-ids should appear in lexigraphically increasing order by their feature name in boh series. For example, if the features "a", "b", "c" had the tf-idfs 1.0, 0.5, 1.0 in the series with the largest tf-idfs, then they should occur in the returned result in the order "a", "c", "b" with values 1.0, 1.0, 0.5.

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    vector = TfidfVectorizer().fit(X_train)
    vector_aux = vector.transform(X_train)
    df_values = pd.DataFrame(vector_aux.todense())
    df_names = pd.DataFrame(vector.get_feature_names())
    #print(df_values)
    df_values.columns = df_names[0]
    #df_values = df_values.reset_index(drop=True)
    a = df_values.max().sort_values()
    t = a.index.rename("")
    smallest_tf_idfs = pd.Series(a[:20], index = t[:20])
    largest_tf_idfs = pd.Series(a[-20:][::-1], index = t[-20:][::-1])
    return (smallest_tf_idfs, largest_tf_idfs)

answer_four()

(
 sympathetic     0.074475
 healer          0.074475
 aaniye          0.074475
 dependable      0.074475
 companion       0.074475
 listener        0.074475
 athletic        0.074475
 exterminator    0.074475
 psychiatrist    0.074475
 pest            0.074475
 determined      0.074475
 chef            0.074475
 courageous      0.074475
 stylist         0.074475
 psychologist    0.074475
 organizer       0.074475
 pudunga         0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64,
 
 146tf150p    1.000000
 havent       1.000000
 home         1.000000
 okie         1.000000
 thanx        1.000000
 er           1.000000
 anything     1.000000
 lei          1.000000
 nite         1.000000
 yup          1.000000
 thank        1.000000
 ok           1.000000
 where        1.000000
 beerage      1.000000
 anytime      1.000000
 too          1.000000
 done         1.000000
 645          1.000000
 tick         0.980166
 blank        0.932702

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [31]:
from sklearn.naive_bayes import MultinomialNB
def answer_five():

    tfidf = TfidfVectorizer(min_df=3).fit(X_train)
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    naive_bayes_classifier = MultinomialNB(alpha=0.1)
    naive_bayes_classifier.fit(X_train_tf, y_train)
    predicted = naive_bayes_classifier.predict_proba(X_test_tf)
    return roc_auc_score(y_test, predicted[:, 1])
answer_five()

0.9954968337775665

In [32]:
from sklearn.naive_bayes import MultinomialNB
tfidf = TfidfVectorizer(min_df=3).fit(X_train)
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)
naive_bayes_classifier = MultinomialNB(alpha=0.1)
naive_bayes_classifier.fit(X_train_tf, y_train)
predicted = naive_bayes_classifier.predict_proba(X_test_tf)
roc_auc_score(y_test, predicted[:, 1])


#Ajuste e transforme os dados de treinamento `X_train` usando um vetorizador Tfidf 
#ignorando termos que tenham uma frequência de documento estritamente inferior a **3**.
#Em seguida, ajuste um modelo classificador multinomial Naive Bayes com suavização `alfa = 0,1` 
#e calcule a pontuação da área sob a curva (AUC) usando os dados de teste transformados.
#*Esta função deve retornar a pontuação AUC como um valor flutuante.*



0.9954968337775665

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [77]:
def answer_six():
   #spam_data['target'].count()
    docSizeSpam = 0
    docSizeNoSpam = 0
    docSizeNoSpamItens = 0
    docSizeSpamItens = 0

    for i, a in spam_data.iterrows():
        if a.target == 1:
            docSizeSpam += len(a.text)
            docSizeSpamItens += 1
        else:
            docSizeNoSpam += len(a.text)
            docSizeNoSpamItens += 1
#print(docSizeSpam/docSizeSpamItens)
#print(docSizeNoSpam/docSizeNoSpamItens)
    return (docSizeNoSpam/docSizeNoSpamItens,docSizeSpam/docSizeSpamItens)
answer_six()


(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [18]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*Hint: Since probability is set to false, use the model's `decision_function` on the test data when calculating the target scores to use in roc_auc_score*

*This function should return the AUC score as a float.*

In [65]:
from sklearn.svm import SVC

def answer_seven():
                          
    length_X_train = list(map(len,X_train))
    length_X_test = list(map(len,X_test))
    tfidf = TfidfVectorizer(min_df=5).fit(X_train)
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    X_train.shape
    X_train_modificado = add_feature(X_train_tf, length_X_train)
    X_test_modificado = add_feature(X_test_tf, length_X_test)
    model = SVC(C=10000).fit(X_train_modificado, y_train)
    predicted = model.decision_function(X_test_modificado)
    return  roc_auc_score(y_test, predicted) 
answer_seven()


0.9963202213809143

### Question 8

What is the average number of digits per document for not spam and spam documents?

*Hint: Use `\d` for digit class*

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [78]:
import re
def answer_eight():
    #spam_data['target'].count()
    docSizeSpam = 0
    docSizeNoSpam = 0
    docSizeNoSpamItens = 0
    docSizeSpamItens = 0

    for i, a in spam_data.iterrows():
        if a.target == 1:
            docSizeSpam += len(re.findall("[0-9]",a.text))
            docSizeSpamItens += 1
        else:
            docSizeNoSpam += len(re.findall("[0-9]",a.text))
            docSizeNoSpamItens += 1
    #print(docSizeSpam)
    #print(docSizeNoSpam)
    return (docSizeNoSpam/docSizeNoSpamItens,docSizeSpam/docSizeSpamItens)
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100` and `max_iter=1000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [103]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    length_X_train = list(map(len,X_train))
    length_X_test = list(map(len,X_test))
    length_number_X_train= [len(re.findall("[0-9]",i)) for i in X_train]
    length_number_X_test= [len(re.findall("[0-9]",i)) for i in X_test]

    tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train).fit(X_train)
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    X_train_modificado = add_feature(X_train_tf, length_X_train)
    X_test_modificado = add_feature(X_test_tf, length_X_test)
    X_train_modificado = add_feature(X_train_modificado, length_number_X_train)
    X_test_modificado = add_feature(X_test_modificado, length_number_X_test)
    model = LogisticRegression(C=100, max_iter=1000).fit(X_train_modificado, y_train)
    predictions = model.decision_function(X_test_modificado)
    return roc_auc_score(y_test, predictions)
answer_nine()

0.9972921582941445

In [102]:
from sklearn.linear_model import LogisticRegression
length_X_train = list(map(len,X_train))
length_X_test = list(map(len,X_test))
length_number_X_train= [len(re.findall("[0-9]",i)) for i in X_train]
length_number_X_test= [len(re.findall("[0-9]",i)) for i in X_test]

tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train).fit(X_train)
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)
X_train_modificado = add_feature(X_train_tf, length_X_train)
X_test_modificado = add_feature(X_test_tf, length_X_test)
X_train_modificado = add_feature(X_train_modificado, length_number_X_train)
X_test_modificado = add_feature(X_test_modificado, length_number_X_test)
model = LogisticRegression(C=100, max_iter=1000).fit(X_train_modificado, y_train)
predictions = model.decision_function(X_test_modificado)
roc_auc_score(y_test, predictions)


0.9972921582941445

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [81]:
def answer_ten():
    #spam_data['target'].count()
    docSizeSpam = 0
    docSizeNoSpam = 0
    docSizeNoSpamItens = 0
    docSizeSpamItens = 0

    for i, a in spam_data.iterrows():
        if a.target == 1:
            docSizeSpam += len(re.findall("\W",a.text))
            docSizeSpamItens += 1
        else:
            docSizeNoSpam += len(re.findall("\W",a.text))
            docSizeNoSpamItens += 1
        #print(docSizeSpam)
        #print(docSizeNoSpam)
    return(docSizeNoSpam/docSizeNoSpamItens,docSizeSpam/docSizeSpamItens)
    

answer_ten()

(17.29181347150259, 29.041499330655956)

In [80]:
#spam_data['target'].count()
docSizeSpam = 0
docSizeNoSpam = 0
docSizeNoSpamItens = 0
docSizeSpamItens = 0

for i, a in spam_data.iterrows():
    if a.target == 1:
        docSizeSpam += len(re.findall("\W",a.text))
        docSizeSpamItens += 1
    else:
        docSizeNoSpam += len(re.findall("\W",a.text))
        docSizeNoSpamItens += 1
    #print(docSizeSpam)
    #print(docSizeNoSpam)
(docSizeNoSpam/docSizeNoSpamItens,docSizeSpam/docSizeSpamItens)

#Qual é o número médio de caracteres que não sejam palavras (qualquer coisa que não seja uma letra, um dígito ou um sublinhado) 
#por documento para documentos que não sejam spam e spam?
#Dica: use as classes de caracteres \w e \W
#Esta função deve retornar uma tupla (média de # caracteres que não sejam palavras, não spam, média de # caracteres que não sejam palavras, spam).

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the **first 2000 rows** of training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100 and max_iter=1000. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [119]:
def answer_eleven():
    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train[:2000])
    length_X_train = list(map(len,X_train))
    length_X_test = list(map(len,X_test))
    length_number_X_train= [len(re.findall("[0-9]",i)) for i in X_train]
    length_number_X_test= [len(re.findall("[0-9]",i)) for i in X_test]
    length_no_caracter_X_train= [len(re.findall("\W",i)) for i in X_train]
    length_no_caracter_X_test= [len(re.findall("\W",i)) for i in X_test]
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    X_train_modificado = add_feature(X_train_tf, length_X_train)
    print(X_train_modificado.shape)
    X_test_modificado = add_feature(X_test_tf, length_X_test)
    X_train_modificado = add_feature(X_train_modificado, length_number_X_train)
    print(X_train_modificado.shape)
    X_test_modificado = add_feature(X_test_modificado, length_number_X_test)
    X_train_modificado = add_feature(X_train_modificado, length_no_caracter_X_train)
    print(X_train_modificado.shape)
    X_test_modificado = add_feature(X_test_modificado, length_no_caracter_X_test)

    print(y_train)
    model = LogisticRegression(C=100, max_iter=1000).fit(X_train_modificado, y_train)
    vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train[:2000])
    predictions = model.predict(X_test_modificado)
    feature_names = np.array(vect.get_feature_names())
    sorted_coef_index = model.coef_[0].argsort()
    small_coefficient = list(feature_names[sorted_coef_index[:10]])
    large_coefficient = list(feature_names[sorted_coef_index[:-11:-1]])
    return (roc_auc_score(y_test, predictions),(small_coefficient,large_coefficient)) 

answer_eleven()

(4179, 3384)
(4179, 3385)
(4179, 3386)
872     0
831     0
1273    1
3314    1
4929    1
       ..
4931    0
3264    1
1653    0
2607    0
2732    0
Name: target, Length: 4179, dtype: int64


(0.9653328353394565,
 (['ber',
   ' las',
   '.cal',
   ' 100 ',
   ' agai',
   'ang',
   '1327 ',
   ' more',
   'ai ',
   ' pla'],
  ['76',
   '123',
   ' lis',
   ' forg',
   'ale ',
   'aig',
   '35',
   '0905',
   '.." ',
   'ateme']))

In [117]:
vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train[:2000])
length_X_train = list(map(len,X_train))
length_X_test = list(map(len,X_test))
length_number_X_train= [len(re.findall("[0-9]",i)) for i in X_train]
length_number_X_test= [len(re.findall("[0-9]",i)) for i in X_test]
length_no_caracter_X_train= [len(re.findall("\W",i)) for i in X_train]
length_no_caracter_X_test= [len(re.findall("\W",i)) for i in X_test]
X_train_tf = tfidf.transform(X_train)
X_test_tf = tfidf.transform(X_test)
X_train_modificado = add_feature(X_train_tf, length_X_train)
print(X_train_modificado.shape)
X_test_modificado = add_feature(X_test_tf, length_X_test)
X_train_modificado = add_feature(X_train_modificado, length_number_X_train)
print(X_train_modificado.shape)
X_test_modificado = add_feature(X_test_modificado, length_number_X_test)
X_train_modificado = add_feature(X_train_modificado, length_no_caracter_X_train)
print(X_train_modificado.shape)
X_test_modificado = add_feature(X_test_modificado, length_no_caracter_X_test)

print(y_train)
model = LogisticRegression(C=100, max_iter=1000).fit(X_train_modificado, y_train)
vect = CountVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train[:2000])
predictions = model.predict(X_test_modificado)
feature_names = np.array(vect.get_feature_names())
sorted_coef_index = model.coef_[0].argsort()
small_coefficient = list(feature_names[sorted_coef_index[:10]])
large_coefficient = list(feature_names[sorted_coef_index[:-11:-1]])
roc_auc_score(y_test, predictions),small_coefficient,large_coefficient 


#Para dizer ao Count Vectorizer para usar n-gramas de caracteres, passe em analyzer='char_wb' que cria n-gramas de caracteres 
#apenas a partir de texto dentro dos limites das palavras. Isso deve tornar o modelo mais robusto a erros ortográficos.
#Usando esta matriz de termos de documento e os seguintes recursos adicionais:
#o comprimento do documento (número de caracteres)
#número de dígitos por documento
#número de caracteres que não sejam palavras (qualquer coisa que não seja uma letra, dígito ou sublinhado).
#ajuste um modelo de Regressão Logística com regularização C=100 e max_iter=1000. Em seguida, calcule a pontuação da área sob
#a curva (AUC) usando os dados de teste transformados.Encontre também os 10 menores e os 10 maiores coeficientes do modelo e 
#retorne-os junto com a pontuação AUC em uma tupla.A lista dos 10 menores coeficientes deve ser classificada como menor primeiro,
#a lista dos 10 maiores coeficientes deve ser classificada como maior primeiro.Os três recursos que foram adicionados à matriz de
#termos do documento deverão ter os seguintes nomes caso apareçam na lista de 
#coeficientes: ['length_of_doc', 'digit_count', 'non_word_char_count']Esta função deve retornar uma tupla
#(pontuação AUC como flutuante, lista de menores coefs, lista de maiores coefs).

(4179, 3384)
(4179, 3385)
(4179, 3386)
872     0
831     0
1273    1
3314    1
4929    1
       ..
4931    0
3264    1
1653    0
2607    0
2732    0
Name: target, Length: 4179, dtype: int64


(0.9653328353394565,
 ['ber',
  ' las',
  '.cal',
  ' 100 ',
  ' agai',
  'ang',
  '1327 ',
  ' more',
  'ai ',
  ' pla'],
 ['76', '123', ' lis', ' forg', 'ale ', 'aig', '35', '0905', '.." ', 'ateme'])

In [ ]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train).fit(X_train)
    X_train_tf = tfidf.transform(X_train)
    X_test_tf = tfidf.transform(X_test)
    X_train_modificado = add_feature(X_train_tf, length_X_train)
    X_test_modificado = add_feature(X_test_tf, length_X_test)
    X_train_modificado = add_feature(X_train_modificado, length_number_X_train)
    X_test_modificado = add_feature(X_test_modificado, length_number_X_test)
    model = LogisticRegression(C=100, max_iter=1000).fit(X_train_modificado, y_train)
    predictions = model.decision_function(X_test_modificado)
    roc_auc_score(y_test, predictions)